# 1: Decision Trees
Decision trees are a powerful and commonly used machine learning technique.

If we wanted to optimize our chances of surviving a bear encounter, we could construct a decision tree to tell us what action to take.As our dataset gets larger though, this gets less and less practical. What if we had 10000 rows and 10 variables? Would you want to look through all the possibilities to construct a tree?

This is where the decision tree machine learning algorithm can help. It enables us to automatically construct a decision tree to tell us what outcomes we should predict in certain situations.

The decision tree algorithm is a supervised learning algorithm -- we first construct the tree with historical data, and then use it to predict an outcome. One of the major advantages of decision trees is that they can pick up nonlinear interactions between variables in the data that linear regression can't.

Trees can be used for classification or regression problems.

In [1]:
import pandas as pd
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
           'marital_status', 'occupation', 'relationship', 'race', 'sex',
           'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'high_income']
income = pd.read_csv('adult.data',names=columns)

In [2]:
print income.head()

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country high_income  
0          2174             0              40   United-States   

# 3: Converting categorical variables
we need to convert the categorical variables in our dataset to numeric variables. This involves assigning a number to each category label, and converting all the labels in a column to numbers.

We can use the Categorical.from_array method from Pandas to perform the conversion to numbers.

In [3]:
col = pd.Categorical.from_array(income['workclass'])
income['workclass'] = col.codes
print(income['workclass'].head())

target_col = ['education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'high_income']

for target in target_col:
    col = pd.Categorical.from_array(income[target])
    income[target] = col.codes

0    7
1    6
2    4
3    4
4    4
Name: workclass, dtype: int8


# 4: Split points
A decision tree is constructed through a series of nodes and branches. A node is where we split the data based on a variable, and a branch is one side of the split. Here's an example:図

This is exactly how a decision tree works -- we keep splitting the data based on variables. As we do this, the tree gets deeper and deeper. The tree we made above is two levels deep, because it has one split, and two "levels" of nodes. 

# 5: Performing a split
In the diagram above, we can split the dataset into two portions based on whether the individual works in the private sector or not.

In [4]:
private_incomes = income[income['workclass'] == 4]
public_incomes = income[income['workclass'] != 4]

# 6: Thinking about the data
When we make a split, some rows will go to the right, and some rows will go to the left. As we build the tree deeper and deeper, each node will "receive" less and less rows.

# 7: Rationale behind splitting
The nodes at the bottom of the tree, when we decide to stop splitting, are called terminal nodes, or leaves.Our goal is to ensure that we can make a prediction on future data. In order to do this, all rows in each leaf must have only one value for our target column.

In order to be able to make this prediction, all leaves need to have rows with only a single value for high_income.

# 8: Entropy
we need a metric for how "together" the different values in the high_income column are.A dataset consisting of only 1s in the high_income column would have low entropy.


Entropy, which is not to be confused with entropy from physics, comes from information theory.Entropy can also be thought of in terms of information. If we flip a coin where both sides are heads, we know upfront that the result will be heads. There's much more complexity, especially when you get to cases with more than two possible outcomes, or differential probabilities.

Entropy can also be thought of in terms of information.

$$-\sum_{i=1}^{c} {\mathrm{P}(x_i) \log_b \mathrm{P}(x_i)}$$

We iterate through each unique value in a single column (in this case, high_income), and assign it to i.

In [7]:
import math

p1 = income[income['high_income']==1].shape[0] / float(income.shape[0])
p2 = income[income['high_income']==0].shape[0] / float(income.shape[0])
income_entropy = -(p1*math.log(p1,2) + p2*math.log(p2,2))
print income_entropy

0.796383955202


# 9: Information gain
We'll need a way to go from computing entropy to figuring out which variable to split based on. We can do this using information gain. Information gain tells us which split will reduce entropy the most.

Here's the formula for information gain:
$$IG(T,A) = Entropy(T)-\sum_{v\in A}\frac{|T_{v}|}{|T|} \cdot Entropy(T_{v})$$

We're computing information gain (IG) for a given target variable (T), and a given variable that we want to split based on (A).  To compute this, we first calculate the entropy for T. Then, for each unique value v in the variable A, we compute the number of rows in which A takes on the value v, and divide it by the total number of rows. We then multiply this by the entropy of the rows where A is v. We add together all of these subset entropies, and subtract from the overall entropy to get information gain.

For an even simpler explanation, we're finding the entropy of each set post-split, weighting it by the number of items in each split, then subtracting from the current entropy. If it's positive, we've lowered entropy with our split. The higher it is, the more we've lowered entropy.

One strategy to construct trees to is to create as many branches at each node as there are unique values for the variable being split on. So if the variable has 3 or 4 values, that would result in 3 or 4 branches. Usually, this is more complexity than it's worth, and doesn't improve prediction accuracy, but it's worth knowing about.

To simplify the calculation of information gain, and to make splits simpler, we won't do it for each unique value. Instead, for the variable we're splitting on, we'll find the median. Any rows where the value of the variable is below the median will split left, and the rest of the rows will split right. To compute information gain, we'll only have to compute entropies for two subsets.例示



In [17]:
import numpy as np

def calc_entropy(column):
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    probabilities = counts/float(len(column))
    
    entropy = 0
    
    for prob in probabilities:
        if prob > 0:
            entropy += prob*math.log(prob,2)
            
    return -entropy

entropy = calc_entropy([1,1,0,0,1])
print entropy

information_gain = entropy - ((.8 * calc_entropy([1,1,0,0])) + (.2 * calc_entropy([1])))
print information_gain

income_entropy = calc_entropy(income['high_income'])

median = income['age'].median()
left_incomes = income[income['age'] <= median]
right_incomes = income[income['age'] > median]
left_prob = left_incomes.shape[0] / float(income.shape[0])

age_information_gain = income_entropy - ((left_prob*calc_entropy(left_incomes['high_income'])) + (1-left_prob)*calc_entropy(right_incomes['high_income']))
print age_information_gain

0.970950594455
0.170950594455
0.0470286613047


# 10: Finding the best split
We'll find the variable to split on by finding which split would have the highest information gain.

In [22]:
def calc_information_gain(data,split_name,target_name):
    #Calculate original entropy.
    original_entropy = calc_entropy(data[target_name])
    
    #Find the median of the column we're spliting
    column = data[split_name]
    median = column.median()
    
    left_split = data[column <= median]
    right_split = data[column > median]
    
    #Loop through the splits, and calculate the subset entropy
    to_subtract = 0
    for subset in [left_split,right_split]:
        prob = (subset.shape[0] / float(data.shape[0]))
        to_subtract += prob * calc_entropy(subset[target_name])
    
    #return information gain
    return original_entropy - to_subtract

print calc_information_gain(income,'age','high_income')

columns = ["age", "workclass", "education_num", "marital_status",
           "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

information_gains = []

for row in columns:
    information_gain = calc_information_gain(income,row,'high_income')
    information_gains.append(information_gain)

highest_gain_index = information_gains.index(max(information_gains))
highest_gain = columns[highest_gain_index]

print information_gains,highest_gain

0.0470286613047
[0.047028661304691965, 0.0068109840543966182, 0.065012984132774232, 0.1114272573715438, 0.0015822303843424645, 0.047362416650269412, 0.0, 0.0, 0.040622468671234868, 0.00013457344495848567] marital_status


# 11: Build the whole tree
We now know how to make a single split. In order to construct a tree, we'll need to construct these splits all the way down the tree, until the leaves only have a single class.図

In order to construct a tree like this, we'll need to be able to split each node multiple times.